In [1]:
import openai
from importlib import reload
from utils.structure_content import process_folder_concurrently,StructuredDocument
from llm_client import llm_client
from llm_client.agent import MultiTurnAgent
from llm_client.prompt_builder import  PromptBuilder
from llm_client.document_vector_store import (
    EmbeddingProcessor,
    Document,
    DocumentStore,
    SimpleDocument,
    VectorStore
)
from kme_doc import KMEDocument
from tqdm import tqdm
import pandas as pd
import os
from content_retrieval import query_document, llm,doc_store, summary_doc_store ,embed,embedding

2025-08-23 11:11:04,800 - INFO - Loading faiss with AVX512 support.
2025-08-23 11:11:04,801 - INFO - Could not load library with AVX512 support due to:
ModuleNotFoundError("No module named 'faiss.swigfaiss_avx512'")
2025-08-23 11:11:04,801 - INFO - Loading faiss with AVX2 support.
2025-08-23 11:11:04,812 - INFO - Successfully loaded faiss with AVX2 support.


Loading .env file from: .env


Loaded FAISS index (61 vectors) and ID set from disk.
Syncing VectorStore with DocumentStore...
VectorStore is already in sync. No new documents to add.


In [2]:
kme_vertaaltabel = pd.read_csv("data/kme_vertaaltabel.csv",sep=';').set_index("KME_ID")

### Verwerken van nieuwe content in content map

In [3]:
belastingsoort= kme_vertaaltabel['BELASTINGSOORT'].unique().tolist()
proces = kme_vertaaltabel['PROCES_ONDERWERP'].unique().tolist()

# Opbouwen Doc store van pdf

In [4]:
def check_for_new_content():
    filenames = [(k,v)[1].metadata['filename'] for k,v in doc_store.documents.items()]
    found_filenames = os.listdir('content')
    new_document_filenames = [filename for filename in found_filenames if filename not in filenames and filename.endswith(".pdf")]
    return new_document_filenames

new_filenames = check_for_new_content()
new_content_doc = {}
if new_filenames:
    new_content_doc = process_folder_concurrently('content',max_workers=8,filenames_to_process=new_filenames)

docs = []
for filename ,doc in new_content_doc.items():
    kme_tax = kme_vertaaltabel.loc[doc.km_number]
    metadata = {'filename':filename,
    'datum':doc.date,
    'BELASTINGSOORT':kme_tax['BELASTINGSOORT'],
    'PROCES_ONDERWERP' : kme_tax['PROCES_ONDERWERP'],
    'PRODUCT_SUBONDERWERP' :kme_tax['PRODUCT_SUBONDERWERP'],
    'VRAAG' : kme_tax['VRAAG'],

    }
    new_doc = Document(doc.km_number,doc.title,doc.full_text,metadata)
    docs.append(new_doc)
doc_store.add(docs)

### Samenvatten en segmenteren content

In [22]:
# dit moet async worden in de toekomst
summary_processor = PromptBuilder(template_path='prompt_templates',name='summarize')
processed_docs = []

for doc_id, doc in tqdm(doc_store.documents.items()):
    if doc_id:#not in summary_doc_store.documents.keys():
        try:
            prompt = summary_processor.create_prompt(document = doc.content,question= doc.metadata['VRAAG'],
                                                    taxonomy_path = [doc.metadata['BELASTINGSOORT'], 
                                                                    doc.metadata['PROCES_ONDERWERP'],
                                                                    doc.metadata['PRODUCT_SUBONDERWERP']])
            output = llm.process(prompt,reasoning_effort='low')
            output_json =output.content
            if summary_processor.verify_json(output_json):
                new_summary = KMEDocument(id = doc_id , title=doc.title,content=output_json['content'],metadata=output_json['metadata'])
                new_summary.metadata.update(doc.metadata)
                new_summary.metadata["full_text"] = doc.content
                processed_docs.append(new_summary)
        except Exception as e:
            continue

if processed_docs:
    summary_doc_store.add(processed_docs)

100%|██████████| 61/61 [03:04<00:00,  3.03s/it]


In [12]:
from content_retrieval import query_document, llm,doc_store, summary_doc_store ,embed,embedding
from llm_client.tools.vector_search_tool import VectorSearchTool
from llm_client.tools.document_shortlist_tool import DocumentShortlistTool # Import the specific tool class
vs_tool = VectorSearchTool(vector_store=embedding)
slt = DocumentShortlistTool()
prompt_processor = PromptBuilder('prompt_templates','search') 
agent = MultiTurnAgent(
    llm_processor=llm,
    prompt_processor=prompt_processor,
    tools=[vs_tool,slt] # Pass the instantiated tool(s)
)

In [13]:
agent.chat(query="Zoek documenten die gaan over het doen van inkomstenbelasting",max_tool_turns=15)

2025-08-23 11:13:08,856 - INFO - HTTP Request: POST http://127.0.0.1:1234/v1/chat/completions "HTTP/1.1 200 OK"


'We need to start plan.<|start|>assistant<|channel|>commentary to=functions.update_scratchpad <|constrain|>json<|message|>{"tasks":[{"task":"Analyseer hoofdvraag en bepaal subvragen","completed":false},{"task":"Bedenk 2-4 subvragen","completed":false},{"task":"Formuleer zoekopdrachten voor elke subvraag","completed":false}]}'

In [24]:
doc = summary_doc_store.documents['KM1000124']

print(doc.metadata)

{'extracted_question': ['Wanneer krijg ik bericht op mijn aangifte?', 'Wij doen ons best om u binnen 6 maanden, na ontvangst van de (eerste) aangifte, een aanslag te sturen. Dit kan een voorlopige of definitieve aanslag zijn.'], 'Tags': ['Erfbelasting', 'Aangifte', 'Afhandelingstermijn', 'Definitieve aanslag', 'Voorlopige aanslag', 'Knelgevallenregeling', 'Versneld behandelen', 'SEA', 'Dienstonderdeel Tridion'], 'filename': 'Wanneer_krijg_ik_bericht_op_mijn_aangifte_KM1000124_v16-0_nl-NL.pdf', 'datum': '17/02/2025', 'BELASTINGSOORT': 'EB - Erfbelasting', 'PROCES_ONDERWERP': 'Aangifte', 'PRODUCT_SUBONDERWERP': 'Aangifte', 'VRAAG': 'Wanneer krijg ik bericht op mijn aangifte?', 'full_text': '\xa0 Wanneer krijg ik bericht op mijn aangifte? \uea7a 17/02/2025 KM1000124 Private information \ue8bc Wanneer bericht Wettelijk gezien zijn we verplicht de aangifte definitief af te handelen binnen 3 jaar na de dag van het overlijden. Wij doen ons  best om u binnen 6 maanden, na ontvangst van de (eer

In [19]:
prompt = summary_processor.create_prompt(document = doc.content,question= doc.metadata['VRAAG'],
                                        taxonomy_path = [doc.metadata['BELASTINGSOORT'], 
                                                        doc.metadata['PROCES_ONDERWERP'],
                                                        doc.metadata['PRODUCT_SUBONDERWERP']])

In [20]:
results = llm.process(prompt)

2025-08-23 11:44:35,077 - INFO - HTTP Request: POST http://127.0.0.1:1234/v1/chat/completions "HTTP/1.1 200 OK"


In [ ]:
results.content

{'content': 'Dit document beschrijft waarom belastingplichtigen een aanmaning ontvangen en welke kosten daarbij komen kijken. De Belastingdienst legt uit dat een aanmaning wordt gestuurd wanneer een aanslag niet, niet volledig of niet op tijd is betaald en dat de aanmaningskosten binnen twee weken na dagtekening voldaan moeten worden. Afhankelijk van het openstaande bedrag bedragen de kosten €9 (tot €454) of €21 (vanaf €454). Het document behandelt ook hoe om te gaan met bezwaren, reeds betaalde situaties en uitstel van betaling. Voor medewerkers is er een gedetailleerde ETM‑instructie over hoe de situatie in het systeem te controleren, klantcontacten te registreren en backofficeberichten op te stellen.',
 'metadata': {'extracted_question': ['Waarom ontvang ik een aanmaning?',
   'U krijgt een aanmaning omdat u de aanslag niet, niet volledig of niet op tijd hebt betaald.'],
  'Tags': ['aanmaning',
   'betaling',
   'kosten',
   'onderdeel: Betalingsverkeer',
   'proces: Betalen',
   'B

In [25]:
doc.metadata

{'extracted_question': ['Wanneer krijg ik bericht op mijn aangifte?',
  'Wij doen ons best om u binnen 6 maanden, na ontvangst van de (eerste) aangifte, een aanslag te sturen. Dit kan een voorlopige of definitieve aanslag zijn.'],
 'Tags': ['Erfbelasting',
  'Aangifte',
  'Afhandelingstermijn',
  'Definitieve aanslag',
  'Voorlopige aanslag',
  'Knelgevallenregeling',
  'Versneld behandelen',
  'SEA',
  'Dienstonderdeel Tridion'],
 'filename': 'Wanneer_krijg_ik_bericht_op_mijn_aangifte_KM1000124_v16-0_nl-NL.pdf',
 'datum': '17/02/2025',
 'BELASTINGSOORT': 'EB - Erfbelasting',
 'PROCES_ONDERWERP': 'Aangifte',
 'PRODUCT_SUBONDERWERP': 'Aangifte',
 'VRAAG': 'Wanneer krijg ik bericht op mijn aangifte?',
 'full_text': '\xa0 Wanneer krijg ik bericht op mijn aangifte? \uea7a 17/02/2025 KM1000124 Private information \ue8bc Wanneer bericht Wettelijk gezien zijn we verplicht de aangifte definitief af te handelen binnen 3 jaar na de dag van het overlijden. Wij doen ons  best om u binnen 6 maanden

In [ ]:
doc_store.search("BELASTINGSOORT:EB",limit=20)

[Document(id='KM1014247', title='Ik heb gezamenlijk aangifte gedaan. Wie krijgt de aanslag?', content='\xa0 Ik heb gezamenlijk aangifte gedaan. Wie krijgt de aanslag? \uea7a 14/11/2022 KM1014247 Private information \ue8bc Aangifte/aanslag meerdere personen Vanaf 2021 Er zijn 2 mogelijkheden 1. Hebt u in de aangifte aangegeven dat de belastingaanslagen naar de executeur testamentair of naar de contactpersoon  gestuurd moeten worden? Dan ontvangt deze persoon alle aanslagen. 2. Is er méér dan 1 erfgenaam aanwezig en hebt u toestemming om namens alle erfgenamen aangifte te doen, dan ontvangt  iedere erfgenaam een eigen aanslag. Is dit niet het geval dan sturen wij de aanslag naar de contactpersoon. Let op! Dit geldt ook als er een beschikking geen aanslag is opgelegd. 2020 en ouder Er zijn 2 mogelijkheden. 1. Hebt u in de aangifte aangegeven dat de belastingaanslagen naar de executeur testamentair of naar de contactpersoon  gestuurd moeten worden? Dan ontvangt deze persoon alle aanslagen.

: 